# 전체코드

# Discriminator와 Generator

In [87]:
from torch import nn

class Discriminator_FCN(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.FCN1 = nn.Linear(784, 100)
        self.FCN2 = nn.Linear(100, 1)
        self.sig = nn.Sigmoid()
        self.LRU = nn.LeakyReLU(0.2)
    def forward(self, z):
        out1 = self.FCN1(z)
        out3 = self.LRU(out1)
        out4 = self.FCN2(out3)
        out5 = self.LRU(out4)
        result = self.sig(out5)
        return result

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.Conv1 = nn.Conv2d(11, 128, 4, 2, 1)
        self.bn1 = nn.BatchNorm2d(128)
        self.Conv2 = nn.Conv2d(128, 256, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(256)
        self.Conv3 = nn.Conv2d(256, 512, 4, 1, 0)
        self.bn3 = nn.BatchNorm2d(512)
        self.Conv4 = nn.Conv2d(512, 1, 4, 1, 0)
        self.LRU = nn.LeakyReLU(0.2)
        self.sig = nn.Sigmoid()

    def forward(self, z):
        z = self.LRU(self.bn1(self.Conv1(z)))
        z = self.LRU(self.bn2(self.Conv2(z)))
        z = self.LRU(self.bn3(self.Conv3(z)))
        out = self.sig(self.LRU(self.Conv4(z)))


        return out

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.DeConv1 = nn.ConvTranspose2d(110, 512, kernel_size=4, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(512)
        self.DeConv2 = nn.ConvTranspose2d(512, 256, kernel_size=4, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(256)
        self.DeConv3 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.DeConv4 = nn.ConvTranspose2d(128, 1, kernel_size=4, stride=2, padding=1)
        self.Relu = nn.ReLU()
        self.tanh = nn.Tanh()

    def forward(self, z):

        out1 = self.Relu(self.bn1(self.DeConv1(z)))
        out2 = self.Relu(self.bn2(self.DeConv2(out1)))
        out3 = self.Relu(self.bn3(self.DeConv3(out2)))
        out4 = self.tanh(self.DeConv4(out3))

        return out4


# 폴더로 부터 이미지 데이터 가져오기

In [88]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
from os.path import join
from os import listdir


class DatasetFromfolder(Dataset):
    def __init__(self, dir_mnist):
        super(DatasetFromfolder, self).__init__()

        self.filelist = []
        self.lenlist = []
        self.lensum = 0
        for i in range(10):
            idir = join(dir_mnist, str(i))
            filelist_tmp = [join(idir, x) for x in listdir(idir)]
            self.filelist.append((filelist_tmp, i))
            self.lenlist.append(len(filelist_tmp))
            self.lensum = self.lensum + len(filelist_tmp)

        self.transform = transforms.Compose([transforms.ToTensor()])

    def __getitem__(self, index):

        c, cindex = self._findlabelfromindex(index)
        clist, label = self.filelist[c]
        resultimage = self.transform(Image.open(clist[cindex]).convert('L'))
        return resultimage, label

    def __len__(self):
        return self.lensum

    def _findlabelfromindex(self, index):
        label = 0
        indexsum = 0

        for i in range(10):
            indexsum += self.lenlist[i]
            if index < indexsum:
                label = i
                break

        classindex = index - indexsum


        return label, classindex


# 트레이닝 데이터로 학습하기

In [202]:
import torch
import torch.nn as nn
#from model import Generator, Discriminator
#from Data_loader import DatasetFromfolder
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import random

onehot = torch.zeros(10,10)
onehot = onehot.scatter_(1, torch.LongTensor([0,1,2,3,4,5,6,7,8,9]).view(10,1),1).view(10,10,1,1)
fill = torch.zeros([10,10,28,28])

for i in range(10):
    fill[i,i,:,:] = 1

NUM_EPOCHS = 100
def train():
    train_set = DatasetFromfolder('./4주차/mnist_png.tar/mnist_png/training')
    train_loader = DataLoader(dataset=train_set, num_workers=0, batch_size=16, shuffle=True)
    
    netG = Generator()
    netD = Discriminator()

    criterion = nn.MSELoss()

    print("train loder 길이 :",len(train_loader))
    
    optimizerD = optim.Adam(netD.parameters(), lr=0.0002)
    optimizerG = optim.Adam(netG.parameters(), lr=0.0002)

    for epoch in range(1, NUM_EPOCHS + 1):
        batch_idx = 0
        for x, label in train_loader:

            batch_size = x.size(0)

            #generator 라벨과 이미지 결합
            x = 2*x - 1
            z = torch.rand(batch_size, 100, 1, 1)
            
            #중복허용 배치사이즈만큼 랜덤함수 뽑기
            number=[random.choice(range(10)) for i in range(batch_size)]

            for i in range(batch_size):
                if i==0:
                    result_onehot = onehot[number[i]].unsqueeze(0)
                    result_fill = fill[number[i]].unsqueeze(0)
                    real_fill = fill[label[i].item()].unsqueeze(0)
                else:
                    result_onehot = torch.cat((result_onehot,onehot[number[i]].unsqueeze(0)),dim=0)
                    result_fill = torch.cat((result_fill, fill[number[i]].unsqueeze(0)),dim=0)
                    real_fill = torch.cat((real_fill, fill[label[i].item()].unsqueeze(0)),dim=0)
            
            #print("1hot : ", result_onehot.size())
            z = torch.cat((z,result_onehot),dim=1)
            
            #print("z = ",z.size())
            fake_image = netG(z)
            
            fake_image = torch.cat((fake_image,result_fill),dim=1)
            real_image = torch.cat((x, real_fill),dim=1)
            #print("x : ",x.size())
            #print("label : ",label.size())

            fake_image = netD(fake_image)
            real_image = netD(real_image)

            netD.zero_grad()
            
            d_loss = criterion(fake_image.squeeze(), torch.zeros(batch_size)) + criterion(real_image.squeeze(), torch.ones(batch_size))
            d_loss.backward(retain_graph=True)
            optimizerD.step()

            netG.train()
            netG.zero_grad()

            g_loss = criterion(fake_image.squeeze(), torch.ones(batch_size))
            g_loss.backward(retain_graph=True)
            optimizerG.step()

            if batch_idx % 20 == 0:
                netG.eval()

                eval_z = torch.rand(batch_size, 110, 1, 1)
                generated_image = netG(eval_z)

                generated_image = (generated_image + 1) / 2

                print("Epoch:{} batch[{}/{}] G_loss:{} D_loss:{}".format(epoch, batch_idx, len(train_loader), g_loss, d_loss))
                torchvision.utils.save_image(generated_image.data, 'result/Generated-%d-%d.png' % (batch_idx, epoch))

            batch_idx += 1


if __name__ == "__main__":
    train()
    # test()




train loder 길이 : 3750
Epoch:1 batch[0/3750] G_loss:0.22240565717220306 D_loss:0.5020285844802856
Epoch:1 batch[20/3750] G_loss:0.29848164319992065 D_loss:0.4301902651786804
Epoch:1 batch[40/3750] G_loss:0.46900054812431335 D_loss:0.10216816514730453
Epoch:1 batch[60/3750] G_loss:0.7097903490066528 D_loss:0.026410581544041634
Epoch:1 batch[80/3750] G_loss:0.519844651222229 D_loss:0.08188030123710632
Epoch:1 batch[100/3750] G_loss:0.5347405672073364 D_loss:0.07455730438232422
Epoch:1 batch[120/3750] G_loss:0.5358700156211853 D_loss:0.09153781831264496
Epoch:1 batch[140/3750] G_loss:0.7022619843482971 D_loss:0.02636188641190529
Epoch:1 batch[160/3750] G_loss:0.7475770711898804 D_loss:0.018538225442171097
Epoch:1 batch[180/3750] G_loss:0.773535430431366 D_loss:0.018132170662283897
Epoch:1 batch[200/3750] G_loss:0.701131284236908 D_loss:0.027055758982896805
Epoch:1 batch[220/3750] G_loss:0.7862741351127625 D_loss:0.014290341176092625
Epoch:1 batch[240/3750] G_loss:0.8326743841171265 D_loss:

FileNotFoundError: [Errno 2] No such file or directory: './4주차/mnist_png.tar/mnist_png/training\\0\\31389.png'

# 테스트하기

In [201]:
def test():
    onehot = torch.zeros(10,10)
    onehot = onehot.scatter_(1, torch.LongTensor([0,1,2,3,4,5,6,7,8,9]).view(10,1),1).view(10,10,1,1)
    fill = torch.zeros([10,10,28,28])
    
    batch_idx=321;epoch=123

    netG = Generator()
    z = torch.rand(batch_size, 100, 1, 1)

    for i in range(10):
        fill[i,i,:,:] = 1

    number=[1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6]
    for i in range(16):
        if i==0:
            result_onehot = onehot[number[i]].unsqueeze(0)
        else:
            result_onehot = torch.cat((result_onehot,onehot[number[i]].unsqueeze(0)),dim=0)

    z = torch.cat((z,result_onehot),dim=1)

    netG.eval()
    generated_image = netG(z)
    generated_image = (generated_image + 1) / 2

    torchvision.utils.save_image(generated_image.data, 'result/Generated-%d-%d.png' % (batch_idx, epoch))
    
    print("generated_image.data : ", generated_image.data)

if __name__ == "__main__":
    #train()
    test()


generated_image.data :  tensor([[[[0.4972, 0.5003, 0.4970,  ..., 0.5002, 0.4971, 0.4940],
          [0.4959, 0.5053, 0.4864,  ..., 0.5052, 0.4859, 0.4924],
          [0.4967, 0.5016, 0.5004,  ..., 0.5021, 0.4992, 0.4961],
          ...,
          [0.4962, 0.5053, 0.4858,  ..., 0.5054, 0.4864, 0.4929],
          [0.4969, 0.5017, 0.4998,  ..., 0.5020, 0.4998, 0.4958],
          [0.4940, 0.5011, 0.4876,  ..., 0.5015, 0.4875, 0.4946]]],


        [[[0.4972, 0.5004, 0.4970,  ..., 0.5002, 0.4970, 0.4941],
          [0.4958, 0.5051, 0.4864,  ..., 0.5053, 0.4858, 0.4924],
          [0.4967, 0.5016, 0.5004,  ..., 0.5022, 0.4997, 0.4961],
          ...,
          [0.4962, 0.5053, 0.4855,  ..., 0.5054, 0.4865, 0.4930],
          [0.4969, 0.5019, 0.4996,  ..., 0.5018, 0.4999, 0.4958],
          [0.4940, 0.5012, 0.4875,  ..., 0.5016, 0.4875, 0.4947]]],


        [[[0.4972, 0.5005, 0.4969,  ..., 0.5003, 0.4970, 0.4941],
          [0.4958, 0.5052, 0.4861,  ..., 0.5055, 0.4863, 0.4924],
          [0.4